# MOS Capacitor

## Capacitance-Voltage (C-V) Analysis

The MOS capacitor is the fundamental building block for understanding MOSFET operation. C-V measurements reveal important information about oxide quality and interface states.

**Learning Objectives:**
- Understand MOS capacitor operation regions
- Simulate C-V characteristics
- Analyze accumulation, depletion, and inversion
- Study flat-band voltage and oxide charges

In [ ]:
# Setup: Load PADRE environment (required on nanoHUB)
# This cell loads the PADRE simulator into your environment.
# If running locally with PADRE already in your PATH, this will be skipped gracefully.

from nanohubpadre import use

# Load the PADRE simulator environment
%use padre-2.4E-r15

print("PADRE environment setup complete.")

---

## Device Parameters Reference

The `describe()` function shows all available parameters for the MOS capacitor factory, including geometry, doping, physical models, and sweep options.

In [ ]:
from nanohubpadre import Simulation

# Show all available parameters for the MOS capacitor
Simulation.describe('mos_capacitor')

---

## 1. MOS Capacitor Physics

### 1.1 Structure

```
    Metal Gate
    ==========
    |  Oxide  | (SiO2)
    ==========
    |         |
    | Silicon | (P-type or N-type)
    |         |
    ==========
    Back Contact
```

### 1.2 Operation Regions (P-type substrate)

- **Accumulation** (Vg < 0): Holes accumulate at surface
- **Flat-band** (Vg = VFB): Bands are flat
- **Depletion** (VFB < Vg < VT): Surface depleted of holes
- **Inversion** (Vg > VT): Electrons form inversion layer

### 1.3 Key Parameters

- **Oxide capacitance**: $C_{ox} = \epsilon_{ox}/t_{ox}$

- **Flat-band voltage**: $V_{FB} = \phi_{ms} - Q_f/C_{ox}$

- **Threshold voltage**: $V_T = V_{FB} + 2\phi_F + Q_d/C_{ox}$

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from nanohubpadre import create_mos_capacitor

# Physical constants
q = 1.6e-19
eps_0 = 8.85e-14  # F/cm
eps_ox = 3.9 * eps_0
eps_si = 11.7 * eps_0
kT = 0.0259  # eV at 300K
ni = 1.5e10  # intrinsic carrier concentration

# MOS parameters
tox = 5e-7   # 5 nm oxide
Na = 1e17    # P-type substrate doping

Cox = eps_ox / tox
phi_F = kT * np.log(Na / ni)

print("MOS Capacitor Parameters:")
print("="*40)
print(f"Oxide thickness: {tox*1e7:.0f} nm")
print(f"Cox = {Cox*1e6:.2f} uF/cm^2")
print(f"phi_F = {phi_F:.3f} V")


---

## 2. Creating a MOS Capacitor Simulation

In [ ]:
# Create MOS capacitor
sim_moscap = create_mos_capacitor(
    # Geometry
    oxide_thickness=0.005,      # 5 nm oxide
    silicon_thickness=0.05,     # 50 nm silicon
    device_width=1.0,
    
    # Doping
    substrate_doping=1e17,      # P-type doping
    substrate_type='p',
    
    # Gate
    gate_type='n_poly',         # N+ polysilicon gate
    
    # Oxide
    oxide_permittivity=3.9,
    
    # Models
    temperature=300,
    
    # Output
    log_bands_eq=True
)

print("MOS Capacitor Configuration:")
print("="*40)
print("Oxide: SiO2, 5 nm")
print("Substrate: P-type Si, 1e17 cm^-3")
print("Gate: N+ polysilicon")

# Run the simulation
print("\nRunning equilibrium simulation...")
result = sim_moscap.run()
if result.returncode == 0:
    print("Simulation completed successfully!")
else:
    print(f"Simulation failed with return code: {result.returncode}")

In [ ]:
# Visualize the MOS capacitor device structure
sim_moscap.device_schematic()

In [ ]:
# View generated deck
print("PADRE Input Deck:")
print("="*60)
print(sim_moscap.generate_deck())

# Plot equilibrium band diagram
print("\n" + "="*60)
print("Equilibrium Band Diagram:")
sim_moscap.plot_band_diagram(title="MOS Capacitor - Equilibrium (Near Flat-band)")

---

## 3. C-V Characteristics

In [ ]:
# C-V simulation with Rappture default parameters
# Matches the nanoHUB MOSCap tool (rappture_moscap.xml) defaults
from nanohubpadre import create_mos_capacitor

sim_cv = create_mos_capacitor(
    # Geometry — Rappture defaults
    oxide_thickness=0.1,         # t_ox = 0.1 um (100 nm)
    ny_oxide=100,                # n_ox = 100 nodes
    silicon_thickness=5.0,       # t_b = 5 um
    ny_silicon=200,              # n_tot = 200 nodes

    # Doping — Rappture default Na = 1e15 /cm3
    substrate_doping=1e15,
    substrate_type='p',

    # Gate — Rappture default: n+ poly silicon
    gate_type='n_poly',

    # Oxide — Rappture default
    oxide_permittivity=3.9,

    # Carrier lifetimes — Rappture default: 1 ns each
    taun0=1e-9,
    taup0=1e-9,

    # Temperature
    temperature=300,

    # HF C-V (1 MHz) — Rappture: freqh = 1e6 Hz
    log_cv=True,
    cv_file='cv_hf',
    ac_frequency=1e6,

    # LF C-V (1 Hz) — Rappture: freql = 1 Hz
    log_cv_lf=True,
    cv_lf_file='cv_lf',
    ac_frequency_lf=1.0,

    # Equilibrium band diagram + quasi-Fermi levels
    # Rappture outputs: band.val, band.con, qfn, qfp at VG=0
    log_bands_eq=True,
    log_qf_eq=True,

    # Equilibrium carrier/potential/E-field profiles
    # Rappture outputs: ele, hole, pot, e.field at VG=0
    log_profiles_eq=True,

    # Gate voltage sweep — Rappture default: v_ini=-3V, v_fin=5V, v_num=100
    vg_sweep=(-3.0, 5.0, 0.08),   # 100 steps over 8V range
)

print('C-V Simulation (Rappture defaults):')
print('='*50)
print(f'  oxide_thickness = 0.1 um (100 nm)')
print(f'  silicon_thickness = 5.0 um')
print(f'  substrate_doping = 1e15 cm^-3 (p-type)')
print(f'  gate_type = n+ poly silicon')
print(f'  taun0 = taup0 = 1 ns')
print(f'  Vg sweep: -3V to +5V, ~100 steps')
print(f'  HF frequency: 1 MHz  |  LF frequency: 1 Hz')

print('\nRunning simulation (HF + LF C-V sweeps)...')
result_cv = sim_cv.run()
if result_cv.returncode == 0:
    print('Simulation completed successfully!')
else:
    print(f'Simulation failed:\n{result_cv.stderr[:500]}')

In [ ]:
# Plot outputs matching the Rappture MOSCap tool tabs
import os
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ── Diagnostics ──────────────────────────────────────────────────────────────
print(f"Return code: {result_cv.returncode}")
print(f"Working dir: {sim_cv.working_dir}")
if os.path.isdir(sim_cv.working_dir):
    files = sorted(os.listdir(sim_cv.working_dir))
    print("Output files:", files)
else:
    print("Working dir not found — simulation did not run.")
    files = []

if result_cv.returncode != 0:
    print("\nSimulation failed — no outputs to plot.")
    # Show last 30 lines of PADRE output for diagnosis
    stdout_lines = result_cv.stdout.strip().splitlines() if result_cv.stdout else []
    if stdout_lines:
        print("--- PADRE output (last 30 lines) ---")
        for line in stdout_lines[-30:]:
            print(line)
else:
    # ── 1. Band Diagram at Equilibrium (Ec, Ev, Efn, Efp) ────────────────────
    print("\n--- Band Diagram at Equilibrium ---")
    try:
        sim_cv.plot_band_diagram(suffix="eq",
                                  title="Band Diagram at Equilibrium (Vg = 0)")
    except Exception as e:
        print(f"  Band diagram: {e}")
        band_files = [f for f in files if f.startswith(('cb', 'vb', 'qfn', 'qfp'))]
        print(f"  Band-related files found: {band_files}")

    # ── 2. Carrier Concentrations at Equilibrium ──────────────────────────────
    print("\n--- Carrier Concentrations at Equilibrium ---")
    try:
        sim_cv.plot_carriers(suffix="eq", log_scale=True,
                              title="Carrier Concentrations at Equilibrium")
    except Exception as e:
        print(f"  Carriers: {e}")

    # ── 3. Electrostatic Potential & E-Field ──────────────────────────────────
    print("\n--- Electrostatics at Equilibrium ---")
    pot = sim_cv.outputs.get('pot_eq')
    ef  = sim_cv.outputs.get('ef_eq')
    if pot is not None and ef is not None and len(pot.x) and len(ef.x):
        fig_pot = make_subplots(rows=1, cols=2,
            subplot_titles=('Electrostatic Potential', 'Electric Field'))
        fig_pot.add_trace(go.Scatter(x=pot.x, y=pot.y, mode='lines',
            line=dict(color='green', width=2), name='Potential'), row=1, col=1)
        fig_pot.add_trace(go.Scatter(x=ef.x, y=ef.y, mode='lines',
            line=dict(color='darkorange', width=2), name='E-field'), row=1, col=2)
        fig_pot.update_xaxes(title_text='Depth (µm)')
        fig_pot.update_yaxes(title_text='Potential (V)', row=1, col=1)
        fig_pot.update_yaxes(title_text='Electric Field (V/cm)', row=1, col=2)
        fig_pot.update_layout(title_text='Electrostatics at Equilibrium (Vg = 0)',
            template='plotly_white', width=1000, height=420)
        fig_pot.show()
    else:
        print(f"  pot_eq={'found' if pot else 'missing'}  "
              f"ef_eq={'found' if ef else 'missing'}")

    # ── 4. C-V Curves: HF and LF ──────────────────────────────────────────────
    print("\n--- C-V Curves ---")
    from nanohubpadre.parser import parse_ac_file

    eps_ox_val = 3.9 * 8.85e-14          # F/cm
    Cox = eps_ox_val / (0.1e-4)          # tox = 0.1 µm = 0.1e-4 cm

    fig_cv = go.Figure()

    # HF C-V
    cv_hf = sim_cv.get_cv_data()
    if cv_hf is not None:
        vg, c = cv_hf.get_cv_data()
        if len(vg):
            fig_cv.add_trace(go.Scatter(x=vg, y=c/Cox, mode='lines',
                line=dict(color='blue', width=2), name='HF C-V (1 MHz)'))
            print(f"  HF C-V: {len(vg)} points, Vg=[{vg[0]:.2f}, {vg[-1]:.2f}] V")
        else:
            print("  HF C-V: file parsed but no data points")
    else:
        hf_files = [f for f in files if 'cv_hf' in f or f == 'cv_hf']
        print(f"  HF C-V: not found  (cv_hf files: {hf_files})")

    # LF C-V
    lf_path = os.path.join(sim_cv.working_dir, 'cv_lf')
    if os.path.exists(lf_path):
        cv_lf = parse_ac_file(lf_path)
        vg_lf, c_lf = cv_lf.get_cv_data()
        if len(vg_lf):
            fig_cv.add_trace(go.Scatter(x=vg_lf, y=c_lf/Cox, mode='lines',
                line=dict(color='red', width=2), name='LF C-V (1 Hz)'))
            print(f"  LF C-V: {len(vg_lf)} points")
        else:
            print("  LF C-V: file found but no data points")
    else:
        print(f"  LF C-V: file not found at {lf_path}")

    if fig_cv.data:
        fig_cv.update_layout(
            title='MOS Capacitor C-V (C/Cox vs Vg)',
            xaxis_title='Gate Voltage Vg (V)',
            yaxis_title='C / Cox',
            template='plotly_white', width=800, height=450,
        )
        fig_cv.show()
    else:
        print("  No C-V data to plot.")

In [ ]:
# Theoretical C-V curve and PADRE simulation comparison
def mos_cv(Vg, Na, tox, VFB=0):
    """Calculate MOS capacitor C-V curve"""
    Cox = eps_ox / tox
    phi_F = kT * np.log(Na / ni)
    
    C = np.zeros_like(Vg)
    
    for i, vg in enumerate(Vg):
        Vgb = vg - VFB
        
        if Vgb < 0:  # Accumulation
            C[i] = Cox
        elif Vgb < 2 * phi_F:  # Depletion
            # Simplified depletion capacitance
            Wd = np.sqrt(2 * eps_si * Vgb / (q * Na))
            Cd = eps_si / Wd
            C[i] = 1 / (1/Cox + 1/Cd)
        else:  # Strong inversion (high frequency)
            Wd_max = np.sqrt(4 * eps_si * phi_F / (q * Na))
            Cd_min = eps_si / Wd_max
            C[i] = 1 / (1/Cox + 1/Cd_min)  # HF: Cmin
    
    return C

Vg = np.linspace(-2, 2, 100)
C_hf = mos_cv(Vg, Na, tox)

fig = go.Figure()

# Theoretical curves
fig.add_trace(go.Scatter(x=Vg, y=C_hf / Cox * 100, mode="lines",
    line=dict(color="blue", width=2, dash="dash"), opacity=0.5,
    name="Theory: High Frequency"))

# Low frequency curve (inversion capacitance = Cox)
C_lf = mos_cv(Vg, Na, tox)
C_lf[Vg > 0.5] = Cox  # LF: inversion charge follows
fig.add_trace(go.Scatter(x=Vg, y=C_lf / Cox * 100, mode="lines",
    line=dict(color="red", width=2, dash="dash"), opacity=0.5,
    name="Theory: Low Frequency"))

# Try to get PADRE C-V data
try:
    cv_data = sim_cv.get_cv_data()
    if cv_data is not None:
        Vg_sim, C_sim = cv_data.get_cv_data()
        # Normalize to Cox
        Cox_val = eps_ox / (tox)
        fig.add_trace(go.Scatter(x=Vg_sim, y=C_sim / Cox_val * 100, mode="lines",
            line=dict(color="green", width=2), name="PADRE Simulation"))
        print("PADRE C-V data extracted successfully!")
except Exception as e:
    print(f"Note: C-V data extraction may require additional output parsing: {e}")
    print("Using theoretical curves for comparison")

fig.update_layout(template="plotly_white", width=1000, height=450,
    title_text="MOS Capacitor C-V Characteristics")
fig.update_xaxes(title_text="Gate Voltage Vg (V)", range=[-2, 2])
fig.update_yaxes(title_text="Capacitance C/Cox (%)", range=[0, 110])

# Add region labels
fig.add_annotation(text="Accumulation", x=-1.5, y=95, showarrow=False, font=dict(size=11))
fig.add_annotation(text="Depletion", x=0, y=60, showarrow=False, font=dict(size=11))
fig.add_annotation(text="Inversion", x=1.5, y=40, showarrow=False, font=dict(size=11))

fig.show()

# Show theoretical parameters
phi_F_val = kT * np.log(Na / ni)
Wd_max = np.sqrt(4 * eps_si * phi_F_val / (q * Na)) * 1e4  # um
Cd_min = eps_si / (Wd_max * 1e-4)
Cmin_Cox = 1 / (1 + Cox / Cd_min)

print("\nTheoretical Parameters:")
print("="*40)
print(f"Surface potential at inversion: 2*phi_F = {2*phi_F_val:.3f} V")
print(f"Maximum depletion width: Wd_max = {Wd_max*1e3:.1f} nm")
print(f"Cmin/Cox = {Cmin_Cox*100:.1f}%")


---

## 4. Effect of Doping

In [ ]:
# Compare different substrate dopings with PADRE simulations
doping_levels = [1e16, 1e17, 1e18]  # cm^-3

print("Effect of Substrate Doping - Running PADRE Simulations:")
print("="*60)

doping_simulations = {}
for Na_val in doping_levels:
    sim = create_mos_capacitor(
        oxide_thickness=0.005,
        substrate_doping=Na_val,
        substrate_type='p',
        log_cv=True,
        log_bands_eq=True,
        vg_sweep=(-2.0, 2.0, 0.1)
    )
    
    print(f"\nNa = {Na_val:.0e} cm^-3:")
    print(f"  Running simulation...")
    result = sim.run()
    if result.returncode == 0:
        print(f"  Completed!")
        doping_simulations[Na_val] = sim
    else:
        print(f"  Failed!")

# Plot theoretical curves
fig = go.Figure()
colors = ["blue", "orange", "green"]

for i, Na_val in enumerate(doping_levels):
    C = mos_cv(Vg, Na_val, tox)
    fig.add_trace(go.Scatter(x=Vg, y=C / Cox * 100, mode="lines",
        line=dict(color=colors[i], width=2, dash="dash"), opacity=0.5,
        name=f"Theory: Na = {Na_val:.0e} cm^-3"))

# Try to plot PADRE results if available
for i, (Na_val, sim) in enumerate(doping_simulations.items()):
    try:
        cv_data = sim.get_cv_data()
        if cv_data is not None:
            Vg_sim, C_sim = cv_data.get_cv_data()
            fig.add_trace(go.Scatter(x=Vg_sim, y=C_sim / Cox * 100, mode="lines",
                line=dict(color=colors[i], width=2),
                name=f"PADRE: Na = {Na_val:.0e} cm^-3"))
    except Exception as e:
        pass  # C-V extraction not available

fig.update_layout(template="plotly_white", width=1000, height=450,
    title_text="Effect of Substrate Doping on C-V")
fig.update_xaxes(title_text="Gate Voltage Vg (V)", range=[-2, 2])
fig.update_yaxes(title_text="Capacitance C/Cox (%)")

fig.show()

print("\nKey observation:")
print("Higher doping -> smaller depletion width -> higher Cmin")


---

## 5. Effect of Oxide Thickness

In [ ]:
# Compare different oxide thicknesses with PADRE simulations
tox_values = [0.002, 0.005, 0.010]  # 2, 5, 10 nm (in um)

print("Oxide Thickness Comparison - Running PADRE Simulations:")
print("="*60)

tox_simulations = {}
for tox_um in tox_values:
    tox_cm = tox_um * 1e-4
    Cox_val = eps_ox / tox_cm
    
    sim = create_mos_capacitor(
        oxide_thickness=tox_um,
        substrate_doping=1e17,
        substrate_type='p',
        log_cv=True,
        log_bands_eq=True,
        vg_sweep=(-2.0, 2.0, 0.1)
    )
    
    print(f"\ntox = {tox_um*1000:.0f} nm: Cox = {Cox_val*1e6:.2f} uF/cm^2")
    print(f"  Running simulation...")
    result = sim.run()
    if result.returncode == 0:
        print(f"  Completed!")
        tox_simulations[tox_um] = sim
    else:
        print(f"  Failed!")

# Plot equilibrium band diagrams for different oxide thicknesses
print("\nEquilibrium Band Diagrams for Different Oxide Thicknesses:")
for tox_um, sim in tox_simulations.items():
    sim.plot_band_diagram(title=f"MOS Capacitor - tox = {tox_um*1000:.0f} nm")

---

## 6. Band Diagrams in Different Regions

In [ ]:
# Simulate band diagrams at different gate voltages
sim_bands = create_mos_capacitor(
    oxide_thickness=0.005,
    substrate_doping=1e17,
    substrate_type='p',
    log_bands_eq=True,
    log_bands_bias=True,
    log_cv=True,
    vg_sweep=(-1.0, 1.5, 0.5)  # Sample points
)

print("Running band diagram simulation at different Vg:")
print("  Vg = -1.0V (Accumulation)")
print("  Vg = -0.5V (Flat-band)")  
print("  Vg = 0.0V (Depletion)")
print("  Vg = 0.5V (Weak inversion)")
print("  Vg = 1.0V (Moderate inversion)")
print("  Vg = 1.5V (Strong inversion)")

result_bands = sim_bands.run()
if result_bands.returncode == 0:
    print("\nSimulation completed!")
    
    # Plot equilibrium band diagram
    print("\nBand Diagram at Equilibrium:")
    sim_bands.plot_band_diagram(title="MOS Capacitor - Band Structure")
else:
    print(f"\nSimulation failed with return code: {result_bands.returncode}")

---

## 7. Complete Simulation Example

In [ ]:
# Complete MOS capacitor characterization
sim_complete = create_mos_capacitor(
    # Geometry
    oxide_thickness=0.005,
    silicon_thickness=0.05,
    device_width=1.0,
    
    # Mesh
    nx=3,
    ny_oxide=15,
    ny_silicon=35,
    
    # Doping
    substrate_doping=1e17,
    substrate_type='p',
    
    # Gate
    gate_type='n_poly',
    
    # Oxide
    oxide_permittivity=3.9,
    oxide_qf=0,  # No fixed charge
    
    # Models
    temperature=300,
    conmob=True,
    fldmob=True,
    
    # Output
    log_cv=True,
    cv_file="moscap_cv",
    log_bands_eq=True,
    
    # C-V sweep
    vg_sweep=(-2.5, 2.5, 0.05),
    ac_frequency=1e6
)

print("Running Complete MOS Capacitor Simulation")
print("="*50)
result_complete = sim_complete.run()

if result_complete.returncode == 0:
    print("Simulation completed successfully!")
    
    # Plot equilibrium band diagram
    print("\nEquilibrium Band Diagram:")
    sim_complete.plot_band_diagram(title="Complete MOS Capacitor Simulation")
    
    # Try to extract and plot C-V data
    try:
        cv_data = sim_complete.get_cv_data()
        if cv_data is not None:
            Vg_sim, C_sim = cv_data.get_cv_data()
            
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=Vg_sim, y=C_sim / Cox * 100, mode="lines",
                line=dict(color="blue", width=2), name="PADRE Simulation"))
            fig.add_trace(go.Scatter(x=Vg, y=C_hf / Cox * 100, mode="lines",
                line=dict(color="red", width=2, dash="dash"), opacity=0.5,
                name="Theory (HF)"))
            fig.update_layout(template="plotly_white", width=1000, height=450,
                title_text="Complete MOS Capacitor C-V Characteristics")
            fig.update_xaxes(title_text="Gate Voltage Vg (V)")
            fig.update_yaxes(title_text="Capacitance C/Cox (%)")
            fig.show()
    except Exception as e:
        print(f"Note: C-V data extraction: {e}")
        
else:
    print(f"Simulation failed with return code: {result_complete.returncode}")
    print("\nGenerated PADRE Input Deck:")
    print(sim_complete.generate_deck())


---

## 8. Double-Gate MOS Capacitor

In a **double-gate** configuration a second gate oxide and gate electrode replace the ohmic back contact.  Both gates couple to the silicon body simultaneously, giving independent control of the top and bottom surfaces.  This geometry is a building block for fully-depleted SOI (FD-SOI) devices and FinFETs.

```
    Gate 1 (top)
    ============
    |  Oxide 1 |  ← top SiO2
    ============
    |  Silicon |  ← thin body (P or N)
    ============
    |  Oxide 2 |  ← bottom SiO2
    ============
    Gate 2 (bottom)
```

Key differences from the single-gate MOS cap:
- **Two coupling oxides** — can use the same or different thicknesses.
- **Electrode 1** = top gate (swept), **Electrode 2** = bottom gate (fixed bias or also swept).
- The body is depleted from both sides simultaneously, which halves the required gate swing.

Use `gate_config="double"` in `create_mos_capacitor` to enable this mode.

In [ ]:
# 8.1 Single-gate vs double-gate: schematic comparison
from nanohubpadre import create_mos_capacitor
from IPython.display import display

# Single-gate (default)
sim_sg = create_mos_capacitor(
    oxide_thickness=0.005,
    silicon_thickness=0.02,
    substrate_doping=1e17,
    substrate_type='p',
    gate_type='n_poly',
    gate_config='single',
)
print('Single-gate structure:')
display(sim_sg.device_schematic())

# Double-gate
sim_dg_sch = create_mos_capacitor(
    oxide_thickness=0.005,
    silicon_thickness=0.02,
    substrate_doping=1e17,
    substrate_type='p',
    gate_type='n_poly',
    gate_config='double',
    back_oxide_thickness=0.005,
    back_gate_type='n_poly',
)
print('Double-gate structure:')
display(sim_dg_sch.device_schematic())

In [ ]:
# 8.2 Double-gate MOS capacitor: structure and C-V
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Double-gate: symmetric oxides, n_poly both gates
sim_dg = create_mos_capacitor(
    oxide_thickness=0.005,       # top oxide: 5 nm
    silicon_thickness=0.02,      # thin body: 20 nm
    substrate_doping=1e17,
    substrate_type='p',
    gate_type='n_poly',
    gate_config='double',
    back_oxide_thickness=0.005,  # bottom oxide: 5 nm (same as top)
    back_gate_type='n_poly',
    log_cv=True,
    cv_file='cv_dg',
    log_bands_eq=True,
    vg_sweep=(-2.0, 2.0, 0.1),
    ac_frequency=1e6,
)

print('Double-gate structure:')
display(sim_dg.device_schematic())

print('\nRunning double-gate simulation...')
result_dg = sim_dg.run()
if result_dg.returncode == 0:
    print('Simulation completed!')
else:
    print('Simulation failed!')
    print(result_dg.stderr)


In [ ]:
# 8.3 Compare equilibrium band diagrams: single-gate vs double-gate

# Single-gate with band logging
sim_sg_bands = create_mos_capacitor(
    oxide_thickness=0.005,
    silicon_thickness=0.02,
    substrate_doping=1e17,
    substrate_type='p',
    gate_type='n_poly',
    gate_config='single',
    log_bands_eq=True,
)
result_sg = sim_sg_bands.run()

# Double-gate with band logging
sim_dg_bands = create_mos_capacitor(
    oxide_thickness=0.005,
    silicon_thickness=0.02,
    substrate_doping=1e17,
    substrate_type='p',
    gate_type='n_poly',
    gate_config='double',
    back_oxide_thickness=0.005,
    back_gate_type='n_poly',
    log_bands_eq=True,
)
result_dg = sim_dg_bands.run()

if result_sg.returncode == 0 and result_dg.returncode == 0:
    print('Both simulations completed.')
    print('\nSingle-gate band diagram:')
    sim_sg_bands.plot_band_diagram(title='Single-Gate MOS Cap — Equilibrium')
    print('\nDouble-gate band diagram:')
    sim_dg_bands.plot_band_diagram(title='Double-Gate MOS Cap — Equilibrium')
else:
    if result_sg.returncode != 0:
        print('Single-gate simulation failed!')
    if result_dg.returncode != 0:
        print('Double-gate simulation failed!')

print('\nKey observation:')
print('In the double-gate structure the band bending occurs at BOTH surfaces')
print('simultaneously, depleting the thin body from top and bottom.')


In [ ]:
# 8.4 Asymmetric double-gate: different top/bottom oxide thicknesses
#     Thicker bottom oxide -> bottom gate has weaker coupling (larger effective tox)

sim_asym = create_mos_capacitor(
    oxide_thickness=0.005,       # top gate: 5 nm (strong coupling)
    silicon_thickness=0.02,
    substrate_doping=1e17,
    substrate_type='p',
    gate_type='n_poly',
    gate_config='double',
    back_oxide_thickness=0.020,  # back gate: 20 nm (4x weaker coupling)
    back_gate_type='n_poly',
    log_bands_eq=True,
)
print('Asymmetric double-gate schematic:')
display(sim_asym.device_schematic())

result_asym = sim_asym.run()
if result_asym.returncode == 0:
    print('\nEquilibrium band diagram (asymmetric):')
    sim_asym.plot_band_diagram(title='Asymmetric Double-Gate — Equilibrium')
else:
    print('Simulation failed!')
    print(result_asym.stderr)


---

## Summary

In this notebook, you learned:

1. **MOS Structure**: Metal-Oxide-Semiconductor stack
2. **Operating Regions**: Accumulation, depletion, inversion
3. **C-V Analysis**: High vs low frequency behavior
4. **Doping Effects**: Higher doping -> higher Cmin
5. **Oxide Thickness**: Thinner oxide -> higher Cox
6. **Double-Gate**: Symmetric gate-oxide-Si-oxide-gate stack; depletes body from both sides

**Key Equations:**
- $C_{ox} = \epsilon_{ox}/t_{ox}$
- $C_{min}/C_{ox}$ depends on $W_d^{max}$ and doping
- Double-gate: $C_{ox,eff}^{-1} = C_{ox,top}^{-1} + C_{ox,bot}^{-1}$ in series

**Next**: [08 - MESFET](08_MESFET.ipynb) - Metal-semiconductor FETs